<a href="https://colab.research.google.com/github/sshee0123/Volleyball_Match_Prediction/blob/main/%EB%B0%B0%EA%B5%AC%EA%B2%BD%EA%B8%B0%EC%98%88%EC%B8%A1_autoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pycaret --quiet
!pip install shap --quiet

     |████████████████████████████████| 320 kB 14.3 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 2.0 MB 43.6 MB/s 
     |████████████████████████████████| 1.7 MB 85.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.8 MB 60.9 MB/s 
     |████████████████████████████████| 167 kB 95.8 MB/s 
     |████████████████████████████████| 88 kB 7.6 MB/s 
     |████████████████████████████████| 1.3 MB 57.2 MB/s 
     |████████████████████████████████| 261 kB 74.9 MB/s 
     |████████████████████████████████| 16.8 MB 59.0 MB/s 
     |████████████████████████████████| 116 kB 88.7 MB/s 
     |████████████████████████████████| 3.1 MB 57.8 MB/s 
     |████████████████████████████████| 10.9 MB 58.2 MB/s 
     |████████████████████████████████| 636 kB 54.4 MB/s 
     |███████████████

In [50]:
!pip install pycaret[full] --quiet

     |████████████████████████████████| 308 kB 13.3 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 305 kB 62.8 MB/s 
     |████████████████████████████████| 100 kB 9.0 MB/s 
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
     |████████████████████████████████| 11.8 MB 62.0 MB/s 
     |████████████████████████████████| 449 kB 52.8 MB/s 
     |████████████████████████████████| 132 kB 50.0 MB/s 
     |████████████████████████████████| 193.7 MB 35 kB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 40 kB 5.5 MB/s 
     |████████████████████████████████| 53.2 MB 119 kB/s 
     |████████████████████████████████| 2.9 MB 55.3 MB/s 
     |████████████████████████████████| 177 kB 93.0 MB/s 
     |████████████████████████████████| 346 kB 64.6 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |██████████████████

# Data load

In [15]:
import pandas as pd
import numpy as np
import datetime

SEED = 42
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/volleyball_data.csv')
data.head()

,Unnamed: 0,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,...,B팀_선수4_범실_범실,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실
0,3,1,29.85,26.83,100.00,0.0,8.33,0.0,45.45,0.20,...,1.0,27.03,30.00,20.0,0.0,0.0,0.0,27.27,0.0,6.0
1,5,0,43.66,34.21,33.33,0.0,58.82,0.0,53.85,0.00,...,3.0,50.00,50.00,0.0,0.0,0.0,0.0,0.00,0.2,1.0
2,6,1,35.29,42.86,25.00,0.0,37.50,0.0,18.18,0.00,...,5.0,17.39,18.18,0.0,0.0,0.0,0.0,18.18,0.0,2.0
3,7,0,54.93,43.24,0.00,0.0,77.78,0.0,40.00,0.00,...,1.0,10.00,20.00,0.0,0.0,0.0,0.0,0.00,0.2,7.0
4,8,1,39.02,29.41,0.00,0.0,56.25,0.0,33.33,0.33,...,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,1.0


In [22]:
# # 결과 열을 제외한 열은 X, 결과 열은 y에 저장
# X = data.drop('결과', axis = 1)
# y = data['결과']

In [23]:
# # 학습, 검증 데이터 분리
# from sklearn.model_selection import train_test_split

# SEED = 42
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state=SEED)

In [43]:
from pycaret.classification import *

In [51]:
from pycaret.classification import models

models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


xgboost
lightgbm

In [45]:
lgbm = create_model('lightgbm', fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5102,0.5067,0.4800,0.5217,0.5000,0.0216,0.0217
1,0.4898,0.5117,0.4400,0.5000,0.4681,-0.0183,-0.0184
2,0.6250,0.6667,0.5833,0.6364,0.6087,0.2500,0.2509
3,0.5833,0.5677,0.6250,0.5769,0.6000,0.1667,0.1672
4,0.5208,0.5503,0.4583,0.5238,0.4889,0.0417,0.0420
Mean,0.5458,0.5606,0.5173,0.5518,0.5331,0.0923,0.0927
Std,0.0504,0.0578,0.0732,0.0493,0.0591,0.1002,0.1005


In [49]:
tuned_lgbm = tune_model(lgbm, fold=5, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5510,0.6000,0.5833,0.5385,0.5600,0.1032,0.1035
1,0.6327,0.6333,0.6250,0.6250,0.6250,0.2650,0.2650
2,0.6667,0.7061,0.5652,0.6842,0.6190,0.3275,0.3322
3,0.6458,0.6574,0.6522,0.6250,0.6383,0.2917,0.2919
4,0.5417,0.5843,0.6522,0.5172,0.5769,0.0912,0.0942
Mean,0.6076,0.6362,0.6156,0.5980,0.6039,0.2157,0.2174
Std,0.0512,0.0432,0.0356,0.0616,0.0300,0.0989,0.0992


In [ ]:
final1_model = finalize_model(tuned_lgbm)
prediction = predict_model(final_model, data=dataset)

In [57]:
# xgb= create_model('xgboost', fold=5)

ValueError: ignored

In [54]:
# xgb= create_model('xgboost', fold=5)
ada = create_model('ada', fold=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6531,0.6467,0.5833,0.6667,0.6222,0.3041,0.3064
1,0.4694,0.5500,0.4167,0.4545,0.4348,-0.0634,-0.0637
2,0.6667,0.6783,0.5652,0.6842,0.6190,0.3275,0.3322
3,0.5833,0.6383,0.6087,0.5600,0.5833,0.1681,0.1687
4,0.5000,0.5339,0.5652,0.4815,0.5200,0.0052,0.0053
Mean,0.5745,0.6094,0.5478,0.5694,0.5559,0.1483,0.1498
Std,0.0792,0.0569,0.0675,0.0934,0.0708,0.1563,0.1579


In [55]:
tuned_ada = tune_model(ada, fold=5, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.5306,0.5067,0.5417,0.5200,0.5306,0.0616,0.0617
1,0.5918,0.5933,0.5417,0.5909,0.5652,0.1820,0.1826
2,0.5625,0.6730,0.4783,0.5500,0.5116,0.1189,0.1198
3,0.5000,0.5391,0.4783,0.4783,0.4783,-0.0017,-0.0017
4,0.5833,0.5635,0.6087,0.5600,0.5833,0.1681,0.1687
Mean,0.5537,0.5751,0.5297,0.5398,0.5338,0.1058,0.1062
Std,0.0341,0.0566,0.0486,0.0382,0.0375,0.0684,0.0686


# 셋업

In [ ]:
# clf = setup(data = train, target = '결과',
#             session_id = SEED,
#             numeric_features=numeric_features,           
#             categorical_features=categorical_features,
#             ignore_features=ignore_features,           
#             categorical_imputation='mode',
#             train_size=0.8,
#             profile = True # 분석을 끝내고 나서 보고서 생성
#             )

In [47]:
setup_clf = setup(data=data, target='결과')

,Description,Value
0,session_id,4938
1,Target,결과
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(347, 92)"
5,Missing Values,False
6,Numeric Features,90
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


In [33]:
top3 = compare_models(sort='Accuracy', n_select=3,fold=5)
top3

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.5585,0.5569,0.5423,0.5627,0.5507,0.1174,0.1180,0.172
ada,Ada Boost Classifier,0.5463,0.5770,0.5103,0.5452,0.5247,0.0939,0.0933,0.102
lightgbm,Light Gradient Boosting Machine,0.5458,0.5606,0.5173,0.5518,0.5331,0.0923,0.0927,0.102
rf,Random Forest Classifier,0.5418,0.5619,0.4853,0.5491,0.5103,0.0854,0.0866,0.532
et,Extra Trees Classifier,0.5211,0.5460,0.5013,0.5289,0.5119,0.0429,0.0440,0.510
ridge,Ridge Classifier,0.5129,0.0000,0.4853,0.5127,0.4974,0.0271,0.0268,0.016
lda,Linear Discriminant Analysis,0.5129,0.4969,0.4937,0.5123,0.5014,0.0271,0.0268,0.016
nb,Naive Bayes,0.5082,0.5331,0.6883,0.5204,0.5820,0.0129,-0.0158,0.014
dt,Decision Tree Classifier,0.5043,0.5047,0.5093,0.4990,0.4965,0.0093,0.0099,0.016
dummy,Dummy Classifier,0.5041,0.5000,1.0000,0.5041,0.6703,0.0000,0.0000,0.012


[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='deviance', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_decrease=0.0, min_impurity_split=None,
                            min_samples_leaf=1, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=100,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=5596, subsample=1.0, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                    n_estimators=50, random_state=5596),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
 

In [34]:
tuned_top3 = [tune_model(i) for i in top3]
tuned_top3 

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6800,0.8077,0.6923,0.6923,0.6923,0.3590,0.3590
1,0.6000,0.5128,0.3846,0.7143,0.5000,0.2138,0.2425
2,0.5417,0.4653,0.5000,0.5455,0.5217,0.0833,0.0836
3,0.6250,0.6528,0.5833,0.6364,0.6087,0.2500,0.2509
4,0.7500,0.8264,0.7500,0.7500,0.7500,0.5000,0.5000
5,0.4583,0.4583,0.5000,0.4615,0.4800,-0.0833,-0.0836
6,0.5833,0.6250,0.8333,0.5556,0.6667,0.1667,0.1925
7,0.5833,0.6528,0.5833,0.5833,0.5833,0.1667,0.1667
8,0.5000,0.5417,0.6667,0.5000,0.5714,0.0000,0.0000


[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.01, loss='deviance', max_depth=7,
                            max_features='log2', max_leaf_nodes=None,
                            min_impurity_decrease=0.01, min_impurity_split=None,
                            min_samples_leaf=4, min_samples_split=2,
                            min_weight_fraction_leaf=0.0, n_estimators=250,
                            n_iter_no_change=None, presort='deprecated',
                            random_state=5596, subsample=0.25, tol=0.0001,
                            validation_fraction=0.1, verbose=0,
                            warm_start=False),
 AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.4,
                    n_estimators=70, random_state=5596),
 LGBMClassifier(bagging_fraction=0.6, bagging_freq=6, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction

In [35]:
blender_top3 = blend_models(estimator_list=tuned_top3, verbose=True, method='auto')
blender_top3

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7200,0.8526,0.7692,0.7143,0.7407,0.4373,0.4387
1,0.5600,0.4551,0.3846,0.6250,0.4762,0.1325,0.1442
2,0.5833,0.5069,0.5833,0.5833,0.5833,0.1667,0.1667
3,0.6667,0.6597,0.6667,0.6667,0.6667,0.3333,0.3333
4,0.8333,0.8889,0.8333,0.8333,0.8333,0.6667,0.6667
5,0.5833,0.5000,0.5833,0.5833,0.5833,0.1667,0.1667
6,0.6250,0.5972,0.8333,0.5882,0.6897,0.2500,0.2750
7,0.6250,0.6528,0.6667,0.6154,0.6400,0.2500,0.2509
8,0.5000,0.5486,0.7500,0.5000,0.6000,0.0000,0.0000


VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.01,
                                                         loss='deviance',
                                                         max_depth=7,
                                                         max_features='log2',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.01,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=4,
                                                         min_samples_split=2,
                                         

In [36]:
blender_tune3 = tune_model(blender_top3)
blender_tune3

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7200,0.8526,0.7692,0.7143,0.7407,0.4373,0.4387
1,0.5600,0.4487,0.3846,0.6250,0.4762,0.1325,0.1442
2,0.5833,0.5069,0.5833,0.5833,0.5833,0.1667,0.1667
3,0.6667,0.6736,0.6667,0.6667,0.6667,0.3333,0.3333
4,0.8333,0.8889,0.8333,0.8333,0.8333,0.6667,0.6667
5,0.5833,0.5069,0.5833,0.5833,0.5833,0.1667,0.1667
6,0.6250,0.5972,0.8333,0.5882,0.6897,0.2500,0.2750
7,0.6250,0.6528,0.6667,0.6154,0.6400,0.2500,0.2509
8,0.5000,0.5486,0.7500,0.5000,0.6000,0.0000,0.0000


VotingClassifier(estimators=[('gbc',
                              GradientBoostingClassifier(ccp_alpha=0.0,
                                                         criterion='friedman_mse',
                                                         init=None,
                                                         learning_rate=0.01,
                                                         loss='deviance',
                                                         max_depth=7,
                                                         max_features='log2',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.01,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=4,
                                                         min_samples_split=2,
                                         

# 모델 예측하기

In [17]:
# final_model = finalize_model(blender_tune3)
# prediction = predict_model(final_model, data=data)
# prediction

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8444,0.844,0.8333,0.8434,0.8383,0.6883,0.6884


,Unnamed: 0,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,...,B팀_선수5_공격종합_성공률,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실,Label
0,3,1,29.85,26.83,100.00,0.0,8.33,0.0,45.45,0.20,...,27.03,30.00,20.0,0.0,0.0,0.00,27.27,0.00,6.0,1
1,5,0,43.66,34.21,33.33,0.0,58.82,0.0,53.85,0.00,...,50.00,50.00,0.0,0.0,0.0,0.00,0.00,0.20,1.0,0
2,6,1,35.29,42.86,25.00,0.0,37.50,0.0,18.18,0.00,...,17.39,18.18,0.0,0.0,0.0,0.00,18.18,0.00,2.0,0
3,7,0,54.93,43.24,0.00,0.0,77.78,0.0,40.00,0.00,...,10.00,20.00,0.0,0.0,0.0,0.00,0.00,0.20,7.0,0
4,8,1,39.02,29.41,0.00,0.0,56.25,0.0,33.33,0.33,...,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,382,1,42.11,33.33,100.00,0.0,38.46,0.0,66.67,0.33,...,30.77,0.00,100.0,0.0,0.0,0.00,75.00,0.25,2.0,1
343,383,1,48.28,50.00,75.00,0.0,60.00,0.0,25.00,0.00,...,50.00,100.00,0.0,0.0,0.0,50.00,0.00,0.00,2.0,0
344,384,0,35.38,37.84,0.00,0.0,26.09,0.0,60.00,0.25,...,55.56,60.00,100.0,0.0,0.0,0.00,33.33,0.25,2.0,0
345,385,0,26.67,0.00,0.00,0.0,0.00,0.0,50.00,0.00,...,20.00,20.00,0.0,0.0,0.0,0.00,0.00,0.25,2.0,0


In [38]:
final_model = finalize_model(blender_tune3)
prediction = predict_model(final_model, data=data.tail(100))
prediction

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.92,0.9815,0.8936,0.9333,0.913,0.839,0.8397


,Unnamed: 0,결과,A팀_선수1_공격종합_성공률,A팀_선수1_오픈_성공률,A팀_선수1_시간차_성공률,A팀_선수1_이동_성공률,A팀_선수1_후위_성공률,A팀_선수1_속공_성공률,A팀_선수1_퀵오픈_성공률,A팀_선수1_서브_성공률,...,B팀_선수5_오픈_성공률,B팀_선수5_시간차_성공률,B팀_선수5_이동_성공률,B팀_선수5_후위_성공률,B팀_선수5_속공_성공률,B팀_선수5_퀵오픈_성공률,B팀_선수5_서브_성공률,B팀_선수5_범실_범실,Label,Score
247,281,1,50.00,45.45,0.0,0.0,57.14,0.0,58.33,0.00,...,50.00,0.00,25.00,0.0,66.67,0.00,0.00,1.0,1,0.5307
248,282,0,40.79,39.39,0.0,0.0,44.44,0.0,33.33,0.00,...,40.00,66.67,33.33,0.0,33.33,0.00,0.00,0.0,0,0.5967
249,283,0,37.84,42.11,0.0,0.0,33.33,0.0,33.33,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.25,1.0,0,0.5945
250,284,0,34.78,26.19,80.0,0.0,39.02,0.0,25.00,0.00,...,25.00,0.00,0.00,0.0,0.00,0.00,0.20,2.0,0,0.5920
251,285,0,24.29,23.68,50.0,0.0,28.57,0.0,11.11,0.00,...,21.05,0.00,0.00,0.0,0.00,40.00,0.20,3.0,1,0.5787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,382,1,42.11,33.33,100.0,0.0,38.46,0.0,66.67,0.33,...,0.00,100.00,0.00,0.0,0.00,75.00,0.25,2.0,1,0.6819
343,383,1,48.28,50.00,75.0,0.0,60.00,0.0,25.00,0.00,...,100.00,0.00,0.00,0.0,50.00,0.00,0.00,2.0,1,0.5630
344,384,0,35.38,37.84,0.0,0.0,26.09,0.0,60.00,0.25,...,60.00,100.00,0.00,0.0,0.00,33.33,0.25,2.0,0,0.6231
345,385,0,26.67,0.00,0.0,0.0,0.00,0.0,50.00,0.00,...,20.00,0.00,0.00,0.0,0.00,0.00,0.25,2.0,0,0.6578


In [39]:
prediction = prediction['Label']
prediction

247    1
248    0
249    0
250    0
251    1
      ..
342    1
343    1
344    0
345    0
346    0
Name: Label, Length: 100, dtype: int64

# 평가

In [40]:
from pycaret.utils import check_metric
check_metric(data['결과'].tail(100), prediction, metric = 'Accuracy')

0.92